# Assignment 17: Kiểm tra đạo văn

Tổng quan: Ở bài tập này chúng ta sẽ lần lượt thực hành các bước để xây dựng một ứng dụng kiểm tra đạo văn sử dụng một mô hình pre-trained word2vec. 

- Dữ liệu được lưu trong hai file: en_source_data.txt chứa văn bản cần kiểm tra, en_target_data.txt chứa các văn bản đối chiếu.
- Bài tập yêu cầu các kiến thức về lập trình Python với các thư viện: gensim (để load word2vec model), numpy (tính similarity). Ngoài ra chúng ta sử dụng một pre-trained word2vec model (sử dụng [Google's pre-trained word2vec model](https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit)).

In [10]:
from gensim.models.keyedvectors import KeyedVectors
import numpy as np

## Câu hỏi 1: khảo sát tập dữ liệu
Hãy lập trình đoạn chương trình dưới đây để khảo sát về số lượng và quan sát một vài ví dụ về bộ dữ liệu.

Gợi ý: 
- Đọc tài liệu cần kiểm tra, từ file "./en_source_data.txt", lưu vào biến source_doc. 
- Đọc các tài liệu đối chiếu từ file "./en_target_data.txt", rồi lưu vào biến target_docs (kiểu list, mỗi doc là một phần tử trong list)


In [11]:
#### YOUR CODE HERE ####

f = open("en_source_data.txt", "r")
source_doc = f.readline()
f. close()

f = open("en_target_data.txt", "r")
target_docs = f.readlines()
f. close()

#### YOUR CODE HERE ####
print("source_doc:\n",source_doc)
print("target_doc_num:", len(target_docs))


source_doc:
 Over the long term, the durability of attitudes toward Trump spotlights the likelihood of a widening rift between two Americas fundamentally diverging in both their exposure to and attitudes about such fundamental dynamics as the nation's growing racial and religious diversity, rising demands for greater racial equality, changing gender roles and the transition from an industrial to an information age economy.

target_doc_num: 23


## Câu hỏi 2: Xây dựng một class cho việc tính document similarity

In [12]:
class DocSim:
    def __init__(self, w2v_model, stopwords=None):
        self.w2v_model = w2v_model
        self.stopwords = stopwords if stopwords is not None else []

    """
    Câu hỏi 2.1:
    Xác định giá trị các vectors cho từng từ trong document. Đầu tiên, tiền
    xử lý tài liệu đầu vào. Sau đó, với mỗi từ ta tính word2vec cho từng từ
    trong văn bản. Cuối cùng, ta tính trung bình các vectors của các từ
    trong văn bản đó. Câu hỏi trong bài tập này là tính giá trị embedding
    của từng từ (cú pháp: self.w2v_model[<từ_cần_tính>]), rồi gán vào biến vec
    """
    def vectorize(self, doc: str) -> np.ndarray:
        doc = doc.lower()
        words = [w for w in doc.split(" ") if w not in self.stopwords]
        word_vecs = []
        for word in words:
            try:
                #### YOUR CODE HERE ####
                
                vec = self.w2v_model[word]
                
                #### YOUR CODE HERE ####
                word_vecs.append(vec)
            except KeyError:
                # Ignore, if the word doesn't exist in the vocabulary
                pass

        vector = np.mean(word_vecs, axis=0)
        return vector

    def _cosine_sim(self, vecA, vecB):
        """
        Câu hỏi 2.2:
        Tính cosine similarity giữa hai vectors (vecA, vecB)
        """
        #### YOUR CODE HERE ####
        
        csim = np.dot(vecA, vecB) / (np.linalg.norm(vecA) * np.linalg.norm(vecB))
        
        #### YOUR CODE HERE ####
        
        if np.isnan(np.sum(csim)):
            return 0
        return csim
        """
        Câu hỏi 2.3:
        Hàm dưới dùng để tính giá trị similarity giữa hai tài liệu cần
        so sánh. Hoàn thành việc tính độ tương tự giữa hai vectors (source_vec,
        target_vec)
        Gợi ý: dùng hàm _cosine_sim()
        """
    def calculate_similarity(self, source_doc, target_docs=None, threshold=0.9):
        """Calculates & returns similarity scores between given source document & all
        the target documents."""
        if not target_docs:
            return []

        if isinstance(target_docs, str):
            target_docs = [target_docs]

        source_vec = self.vectorize(source_doc)
        results = []
        for doc in target_docs:
            target_vec = self.vectorize(doc)
            #### YOUR CODE HERE ####
            
            sim_score = self._cosine_sim(source_vec, target_vec)
            
            #### YOUR CODE HERE ####
            if sim_score > threshold:
                results.append({"score": sim_score, "doc": doc})
            # Sort results by score in desc order
            results.sort(key=lambda k: k["score"], reverse=True)

        return results

## Câu hỏi 3: Tạo đối tượng
Sau khi đã tạo xong lớp để tính toán similarity giữa các docs (DocSim), chúng ta đi tạo đối tượng cho lớp trên:

- Load pre-trained word embedding model
- Tạo danh sách từ dừng
- Sau cùng là tạo đối tượng cho lớp DocSim

Gợi ý: tạo đối tượng ds của lớp DocSim() với hai tham số là model và stopwords=stopwords

In [13]:
googlenews_model_path = '/Users/TheHV/Downloads/GoogleNews-vectors-negative300.bin'
stopwords_path = './stopwords_en.txt'
model = KeyedVectors.load_word2vec_format(googlenews_model_path, binary=True)
with open(stopwords_path, 'r') as fh:
    stopwords = fh.read().split(",")
#### YOUR CODE HERE ####

ds = DocSim(model,stopwords=stopwords)

#### YOUR CODE HERE ####

## Câu hỏi 4: Kiểm tra đạo văn cho một doc mới
Gợi ý: Sử dụng hàm ds.calculate_similarity() để tính độ tương tự gữa tài liệu cần kiểm tra source_doc, và các tài liệu đối chiếu (target_docs)

In [14]:
#### YOUR CODE HERE ####

sim_scores = ds.calculate_similarity(source_doc, target_docs)

#### YOUR CODE HERE ####
print(sim_scores)

[{'score': 1.0, 'doc': "Over the long term, the durability of attitudes toward Trump spotlights the likelihood of a widening rift between two Americas fundamentally diverging in both their exposure to and attitudes about such fundamental dynamics as the nation's growing racial and religious diversity, rising demands for greater racial equality, changing gender roles and the transition from an industrial to an information age economy.\n"}]
